In [79]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)
print(np.__version__)

  Using cached https://files.pythonhosted.org/packages/b0/ee/5ff445dd43b9820e5494d21240e689d3b7cb52bc93f4f164eba84206cd0d/numpy-1.17.4-cp36-cp36m-win_amd64.whl
1.13.1
1.15.4


keras 2.2.0 has requirement keras-applications==1.0.2, but you'll have keras-applications 1.0.7 which is incompatible.
keras 2.2.0 has requirement keras-preprocessing==1.0.1, but you'll have keras-preprocessing 1.0.9 which is incompatible.


In [48]:
# In CSV, label is the first column, after the features, followed by the key
df_train = pd.read_csv('./ieee_train_corrs_removed.csv')
df_test = pd.read_csv('./ieee_test_corrs_removed.csv')

print(df_train.shape)
print(df_test.shape)

(590540, 85)
(506691, 84)


In [65]:
df_train = df_train.astype('float32')
df_test = df_test.astype('float32')

In [66]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Data columns (total 84 columns):
TransactionID     506691 non-null float32
TransactionAmt    506691 non-null float32
ProductCD         506691 non-null float32
card1             506691 non-null float32
card2             506691 non-null float32
card3             506691 non-null float32
card4             506691 non-null float32
card5             506691 non-null float32
card6             506691 non-null float32
addr1             506691 non-null float32
dist1             506691 non-null float32
P_emaildomain     506691 non-null float32
R_emaildomain     506691 non-null float32
C1                506691 non-null float32
C5                506691 non-null float32
D1                506691 non-null float32
D2                506691 non-null float32
D4                506691 non-null float32
D5                506691 non-null float32
D6                506691 non-null float32
D8                506691 non-null float32
D10    

In [67]:
train_data, val_data = np.split(df_train.sample(frac = 1), [int(0.9*len(df_train))])
print(train_data.shape)
print(val_data.shape)

FEATURES = train_data.drop(columns = ['isFraud'])
LABEL = 'isFraud'

(531486, 85)
(59054, 85)


In [68]:
val_data.head()

,TransactionID,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,meta_V176_sum,meta_V187_sum,meta_V188_sum,meta_V221_sum,meta_V228_sum,meta_V229_sum,meta_V282_sum,meta_V283_sum,meta_V288_sum,meta_id_01_sum
417449,3404449.0,0.0,57.939999,4.0,17598.0,268.0,150.0,4.0,166.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
167334,3154334.0,0.0,68.500000,4.0,2157.0,111.0,150.0,4.0,226.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
322425,3309425.0,0.0,107.940002,4.0,3821.0,111.0,150.0,2.0,219.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303827,3290827.0,0.0,57.939999,4.0,15240.0,369.0,150.0,2.0,224.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135265,3122265.0,0.0,51.000000,4.0,4090.0,490.0,150.0,4.0,226.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## TensorFlow Input Functions to read Pandas Dataframe

In [69]:
def train_input_fn(df, n_epochs):
  return tf.estimator.inputs.pandas_input_fn(
      x = df,
      y = df[LABEL],
      batch_size = 128,
      num_epochs = n_epochs,
      shuffle = True,
      queue_capacity = 1000
      )

def val_input_fn(df, n_epochs):
  return tf.estimator.inputs.pandas_input_fn(
      x = df,
      y = df[LABEL],
      batch_size = 128,
      num_epochs = n_epochs,
      shuffle = False,
      queue_capacity = 1000
  )

def pred_input_fn(df, n_epochs):
  return tf.estimator.inputs.pandas_input_fn(
      x = df,
      y = None,
      batch_size = 128,
      shuffle = False,
      queue_capacity = 1000
  )
  

## Feature columns for tf.estimator

In [70]:
def create_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

## Logistic Regression with tf.estimator Framework

In [71]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'ieee_fraud_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearClassifier(
      feature_columns = create_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = train_input_fn(train_data, n_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'ieee_fraud_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FF8A5A45C0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Gr

TypeError: Fetch argument 0 has invalid type <class 'numpy.int64'>, must be a string or Tensor. (Can not convert a int64 into a Tensor or Operation.)

In [80]:
train_data.dtypes

TransactionID     float32
isFraud           float32
TransactionAmt    float32
ProductCD         float32
card1             float32
card2             float32
card3             float32
card4             float32
card5             float32
card6             float32
addr1             float32
dist1             float32
P_emaildomain     float32
R_emaildomain     float32
C1                float32
C5                float32
D1                float32
D2                float32
D4                float32
D5                float32
D6                float32
D8                float32
D10               float32
D11               float32
M1                float32
M4                float32
M6                float32
M7                float32
V95               float32
V99               float32
                   ...   
meta_D15_sum      float32
meta_V10_sum      float32
meta_V12_sum      float32
meta_V19_sum      float32
meta_V29_sum      float32
meta_V35_sum      float32
meta_V37_sum      float32
meta_V39_sum

In [81]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531486 entries, 116460 to 157498
Data columns (total 85 columns):
TransactionID     531486 non-null float32
isFraud           531486 non-null float32
TransactionAmt    531486 non-null float32
ProductCD         531486 non-null float32
card1             531486 non-null float32
card2             531486 non-null float32
card3             531486 non-null float32
card4             531486 non-null float32
card5             531486 non-null float32
card6             531486 non-null float32
addr1             531486 non-null float32
dist1             531486 non-null float32
P_emaildomain     531486 non-null float32
R_emaildomain     531486 non-null float32
C1                531486 non-null float32
C5                531486 non-null float32
D1                531486 non-null float32
D2                531486 non-null float32
D4                531486 non-null float32
D5                531486 non-null float32
D6                531486 non-null float32
D8